In [1]:
# import sys
# !conda install --yes --prefix {sys.prefix} pingouin

In [2]:
import datetime
import numpy as np
import math
import os
import pandas as pd
import pingouin as pg
import re
import seaborn as sns
import json
import statsmodels.api as sm 
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import sys

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    

os.chdir(r'C:\Users\User\GitHub\WebET_Analysis')
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis


In [3]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Raw Data

## Search for specific subjects

In [4]:
#path = 'data_cognition'
#subject_files = os.listdir(path)
#for i in range(0, len(subject_files)):
#    thisSubject = open(path + "/" + subject_files[i]).read()
#    if thisSubject.find('5c0e4e3d87e876000151cfec') > (-1):
#        print(subject_files[i])

## Read CSV from String

In [5]:
def cleanhtml(raw_html):
    # https://stackoverflow.com/questions/9662346/python-code-to-remove-html-tags-from-a-string
    cleanr = re.compile('<.*?>')
    cleanText = re.sub(cleanr, '', raw_html)
    return cleanText

def cleanETText(text):
    textWithinBrackets = re.findall(re.compile('\[.*?\]'), text)
    output = text
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = re.sub(",", "$", old)
        output = output.replace(old, new)
    return output

def cleanSurveyText(text):
    output = text
    textWithinBrackets = re.findall(re.compile('\{.*?\}'), text)
    for i in range(0,len(textWithinBrackets)):
        old = textWithinBrackets[i]
        new = old.replace(',', '§')
        output = output.replace(old, new)
    return output

def compileData(path):
    subject_files = os.listdir(path)
    all_subjects = []
    for i in range(0, len(subject_files)):
        csv_thisSubject = open(path + "/" + subject_files[i]).read()
        csv_thisSubject = cleanhtml(csv_thisSubject)
        csv_thisSubject = cleanETText(csv_thisSubject)
        csv_thisSubject = cleanSurveyText(csv_thisSubject)
        all_subjects.append(pd.read_csv(StringIO(csv_thisSubject)))
    output = pd.concat(all_subjects).reset_index(drop=True)
    return output

data_raw = compileData("data_prolific")

# Custom modifications
data_raw['run_id'].unique()

array([1, 103, 105, 106, 108, 11, 117, 12, 121, 124, 125, 126, 128, 13,
       130, 131, 14, 19, 24, 25, 28, 3, 30, 32, 36, 37, 38, 4, 41, 42, 43,
       45, 47, 48, 49, 5, 50, 51, 54, 56, 57, 58, 59, 6, 61, 63, 66, 67,
       7, 70, 72, 74, 75, 80, 81, 83, 85, 88, 9, 90, 91, 92, 93, 94, 96,
       97, 98, 99], dtype=object)

In [6]:
data_yang = compileData("data_yang2020WG")
# Custom modifications
data_yang.loc[0:515, 'run_id'] = 0
data_yang = data_yang.loc[~data_yang['run_id'].isin([4, 18]), :]
data_yang = data_yang.rename(columns={'eyeData': 'et_data'})
data_yang['run_id'] = data_yang['run_id'] + 1000
print(data_yang['run_id'].unique())
data_raw = data_raw.append(data_yang)

[1000 1001 1011 1013 1014 1015 1016 1017 1019 1002 1020 1021 1003 1007
 1008 1009 1022 1024]


In [7]:
data_milieu = compileData("data_milieu")
data_milieu['run_id'] = data_milieu['run_id'] + 2000
print(data_milieu['run_id'].unique())
data_raw = data_raw.append(data_milieu)

[2011 2012 2013 2002 2008 2009]


### Exclude empty studies

In [8]:
data_raw = data_raw.loc[data_raw['trial_index']>0, :]

### survey data

In [9]:
def cleanOptionalNote(text):
    optionalNoteText = re.findall(re.compile('optionalNote":.*?\}'), text)
    if len(optionalNoteText) < 1:
        output = text
    else:
        old = optionalNoteText[0]
        new = old.replace('§', ' ')
        output = text.replace(old, new)
    return output


def surveyStringToFrame(subject, string):
    string = cleanOptionalNote(string)
    string = re.sub("""{""", '', string)
    string = re.sub("""}""", '', string)
    string = re.sub('"', '', string)
    string = re.sub('§', '$', string)
    output = pd.read_csv(StringIO(string),
                         sep=":",
                         lineterminator="$",
                         header=None,
                         index_col=0
                         ).transpose()
    return output


def surveyData_thisSubject(data):
    df_thisSubject = data.loc[
        (pd.notna(data["responses"])) &
        (data["responses"] != '"'), :].reset_index()
    subject = df_thisSubject['run_id'].unique()[0]
    output = []
    for i in range(0, len(df_thisSubject)):
        output.append(
            surveyStringToFrame(subject,
                                df_thisSubject.loc[i, 'responses'])
        )

    output = pd.concat(output, axis=1)
    output['run_id'] = subject
    return output

def addSurveyData(data):
    surveyData_allSubjects = pd.DataFrame(columns=[
            'prolificID', 'age', 'gender', 'ethnic', 'sight', 
            'glasses', 'degree', 'eyeshadow', 'masquara', 'eyeliner', 
            'browliner', 'vertPosition', 'triedChin', 'keptHead', 
            'optionalNote', 'run_id'])

    for subject in data['run_id'].unique():
        surveyData_allSubjects = \
            surveyData_allSubjects.append(
                surveyData_thisSubject(
                    data.loc[data['run_id']==subject, ['run_id', 'responses']]
                )
            )
    # print(surveyData_allSubjects)
    
    data = data.merge(surveyData_allSubjects, on='run_id', how='left')
    data = data.drop(columns='responses')
    return(data)
    
data_raw = addSurveyData(data_raw)
print(data_raw['run_id'].unique())
print(data_raw.columns)

[1 103 106 108 11 12 124 125 126 128 13 130 131 14 19 24 25 28 30 32 36 37
 38 4 41 42 43 45 47 48 49 5 54 56 58 59 6 61 63 66 67 7 70 72 74 75 80 81
 83 85 88 9 90 91 92 93 94 96 97 98 99 1000 1001 1011 1014 1015 1016 1017
 1019 1002 1020 1021 1003 1007 1008 1009 1022 1024 2011 2012 2013 2002
 2008 2009]
Index(['run_id', 'condition', 'rt', 'stimulus', 'key_press', 'trial_type',
       'trial_index', 'time_elapsed', 'internal_node_id', 'subject',
       'chinFirst', 'choiceTask_amountLeftFirst', 'webcam_label', 'webcam_fps',
       'webcam_height', 'webcam_width', 'button_pressed', 'window_width',
       'window_height', 'chin', 'success', 'x_pos', 'y_pos', 'task_nr',
       'et_data', 'trial_duration', 'option_topLeft', 'option_bottomLeft',
       'option_topRight', 'option_bottomRight', 'recorded_at', 'ip',
       'user_agent', 'device', 'browser', 'browser_version', 'platform',
       'platform_version', 'Unnamed: 2', 'chosenAmount', 'chosenDelay',
       'webcam_Id', 'webcam_aspect

In [10]:
def convertToNumeric(data, columns):
    data[columns] = data[columns].apply(pd.to_numeric, errors='coerce')
    return data


data_raw = convertToNumeric(data_raw, ['age'])
data_raw['degree'] = data_raw['degree'].replace(
        [
            'College / Undergraduate / Bachelor',
            'High School',
            'Graduate / PhD / Master',
            'Middle School'
        ],
    [
            'college',
            'highSchool',
            'grad',
            'middle'
        ]
)

columns = [
            'chinFirst',
            'eyeshadow', 
            'masquara',
            'eyeliner',
            'browliner',
            'triedChin', 
            'keptHead',
        ]

data_raw[columns] = data_raw[columns].replace({'no': 0, 'yes': 1}) 

data_raw['run_id'].unique()

array([1, 103, 106, 108, 11, 12, 124, 125, 126, 128, 13, 130, 131, 14, 19,
       24, 25, 28, 30, 32, 36, 37, 38, 4, 41, 42, 43, 45, 47, 48, 49, 5,
       54, 56, 58, 59, 6, 61, 63, 66, 67, 7, 70, 72, 74, 75, 80, 81, 83,
       85, 88, 9, 90, 91, 92, 93, 94, 96, 97, 98, 99, 1000, 1001, 1011,
       1014, 1015, 1016, 1017, 1019, 1002, 1020, 1021, 1003, 1007, 1008,
       1009, 1022, 1024, 2011, 2012, 2013, 2002, 2008, 2009], dtype=object)

# data_trial

In [11]:
data_trial = data_raw.loc[
    :,
    [
        'run_id', 'subject', 'prolificID', 'chinFirst', 
        'trial_index',
        'trial_type', 'task_nr', 
        'rt', 'stimulus', 'key_press', 
        'time_elapsed', 'trial_duration', 'recorded_at',
        'window_width', 'window_height', 'success', 

        'chin', 'x_pos', 'y_pos', 
        
        'choiceTask_amountLeftFirst', 
        'option_topLeft', 'option_bottomLeft',
        'option_topRight', 'option_bottomRight', 
        'chosenAmount', 'chosenDelay',
    ]
]

## Clean

### Failed Webgazer Setup

In [12]:
subjects_failedSetups = []
for subject in data_trial['run_id'].unique():
    maxTrialIndex = data_trial.loc[data_raw['run_id']==subject, ['trial_index']].max()
    if int(maxTrialIndex) < 15:
        subjects_failedSetups.append(subject)

data_trial.loc[data_trial['run_id'].isin(subjects_failedSetups), 
    [
        'run_id', 'prolificID'
    ]
].drop_duplicates()    

,run_id,prolificID
1036,106,5d485e8415055400194b707f
1038,108,55b237e6fdf99b19ea79d2f7
2079,124,56b7a271e77ebe000bbeff49
4684,14,5ec2cdfd1a17930ddf6f1443
12984,49,5ecfc227f036c902457fc44c
13507,54,5f94d146f65f56336a61cc16
18338,72,600324757b530e39903effc0
20414,83,600324757b530e39903effc0
20935,88,5ecfc227f036c902457fc44c
25608,99,55b237e6fdf99b19ea79d2f


In [13]:
data_trial = data_trial.loc[~data_trial['run_id'].isin(subjects_failedSetups), :]
print(data_trial['run_id'].unique())

[1 103 11 12 125 126 128 13 130 131 19 24 25 28 30 32 36 37 38 4 41 42 43
 45 47 48 5 56 58 59 6 61 63 66 67 7 70 74 75 80 81 85 9 90 91 92 93 94 96
 97 98 1000 1011 1021 1003 1008 2011 2012 2013 2002 2008 2009]


### Duplicate Prolific ID's

In [14]:
data_trial.loc[data_trial['run_id']==2, 'prolificID'] = 'Tim2'


duplicates = data_trial.loc[:, ['prolificID', 'trial_index']].duplicated()
duplicateSubjects = data_trial.loc[duplicates, 'run_id'].unique()

if len(duplicateSubjects) > 0:
    print('! Attention: Duplicate subjects: Check out the following: \n')
    print(duplicateSubjects)
else:
    print('Success: No duplicate subjects found')

! Attention: Duplicate subjects: Check out the following: 

[2002]


### Choice data

In [15]:
# Cleaning
data_trial.loc[: , 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ] = data_trial.loc[:, 
       [
           'option_topLeft',
           'option_bottomLeft', 
           'option_topRight', 
           'option_bottomRight', 
           'chosenAmount', 
           'chosenDelay'
       ] 
    ].replace(['Today', 'Tomorrow', '7 days', '15 days', '30 days', '90 days', '180 days'], 
             [0, 1, 7, 15, 30, 90, 180]) \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) 

### Numerics

In [16]:
data_trial = convertToNumeric(data_trial, 
    [
        'run_id', 'subject', 'chinFirst', 'chin', 'task_nr', 'trial_index', # Int
        'key_press', 
        'x_pos', 'y_pos', 'time_elapsed', 'trial_duration',
        'rt',
        'window_width', 'window_height', 
        'option_topLeft',
        'option_bottomLeft', 
        'option_topRight', 
        'option_bottomRight', 
        'chosenAmount', 
        'chosenDelay'
    ]
)

# data_et

## Compile

In [17]:
def reformatYang(text):
    text = text.replace('$', ',')
    text = text.replace("relative-x", "x")
    text = text.replace("relative-y", 'y')
    text = text.replace('elapse-time', 't')
    text = (text[11:len(text)-1])
    return(text)

for subject in [0, 11000, 21000, 3000, 8000]:
    for i in data_raw.loc[(data_raw['run_id']==subject) &
                      (data_raw['et_data']!='"'), :].index:
        data_raw.loc[i, 'et_data'] = reformatYang(data_raw.loc[i, 'et_data'])

In [ ]:
def textToDataframe(text):
    text = text.replace('$', ',')
    dataframe = pd.read_json(text, orient='records')
    return(dataframe)

def extractEyetrackingData(data):
    data_eyetracking = pd.DataFrame(columns=['x', 'y', 't'])
    data["et_data"] = data['et_data'].apply(str)

    for i in data.loc[~(data['et_data'].isin(['"', '[]', 'nan'])), :].index:
        print('Start decoding Index ' + str(i))
        df = textToDataframe(data.loc[i, 'et_data'])        

        df["t_task"] = (df.loc[:, "t"] - df.loc[0, "t"])
        df['run_id'] = data.loc[i, 'run_id']
        df['trial_index'] = data.loc[i, 'trial_index']

        data_eyetracking = data_eyetracking.append(pd.DataFrame(data=df), ignore_index=True)
        print('Index ' + str(i) + ' extracted')
    return(data_eyetracking)

data_et = extractEyetrackingData(data_raw.loc[data_raw['run_id'].isin(data_trial['run_id'].unique()), :])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Start decoding Index 17
Index 17 extracted
Start decoding Index 19
Index 19 extracted
Start decoding Index 21
Index 21 extracted
Start decoding Index 23
Index 23 extracted
Start decoding Index 25
Index 25 extracted
Start decoding Index 27
Index 27 extracted
Start decoding Index 29
Index 29 extracted
Start decoding Index 31
Index 31 extracted
Start decoding Index 33
Index 33 extracted
Start decoding Index 35
Index 35 extracted
Start decoding Index 37
Index 37 extracted
Start decoding Index 39
Index 39 extracted
Start decoding Index 41
Index 41 extracted
Start decoding Index 43
Index 43 extracted
Start decoding Index 45
Index 45 extracted
Start decoding Index 47
Index 47 extracted
Start decoding Index 49
Index 49 extracted
Start decoding Index 51
Index 51 extracted
Start decoding Index 53
Index 53 extracted
Start decoding Index 55
Index 55 extracted
Start decoding Index 57
Index 57 extracted
Start decoding Index 59
Index 59 extracted
Start decoding Index 61
Index 61 extracted
Start decod

Index 354 extracted
Start decoding Index 355
Index 355 extracted
Start decoding Index 357
Index 357 extracted
Start decoding Index 358
Index 358 extracted
Start decoding Index 360
Index 360 extracted
Start decoding Index 361
Index 361 extracted
Start decoding Index 363
Index 363 extracted
Start decoding Index 364
Index 364 extracted
Start decoding Index 366
Index 366 extracted
Start decoding Index 367
Index 367 extracted
Start decoding Index 369
Index 369 extracted
Start decoding Index 370
Index 370 extracted
Start decoding Index 372
Index 372 extracted
Start decoding Index 373
Index 373 extracted
Start decoding Index 375
Index 375 extracted
Start decoding Index 376
Index 376 extracted
Start decoding Index 378
Index 378 extracted
Start decoding Index 379
Index 379 extracted
Start decoding Index 381
Index 381 extracted
Start decoding Index 382
Index 382 extracted
Start decoding Index 393
Index 393 extracted
Start decoding Index 395
Index 395 extracted
Start decoding Index 397
Index 397 

Index 789 extracted
Start decoding Index 790
Index 790 extracted
Start decoding Index 792
Index 792 extracted
Start decoding Index 793
Index 793 extracted
Start decoding Index 795
Index 795 extracted
Start decoding Index 796
Index 796 extracted
Start decoding Index 798
Index 798 extracted
Start decoding Index 799
Index 799 extracted
Start decoding Index 801
Index 801 extracted
Start decoding Index 802
Index 802 extracted
Start decoding Index 804
Index 804 extracted
Start decoding Index 805
Index 805 extracted
Start decoding Index 807
Index 807 extracted
Start decoding Index 808
Index 808 extracted
Start decoding Index 810
Index 810 extracted
Start decoding Index 811
Index 811 extracted
Start decoding Index 813
Index 813 extracted
Start decoding Index 814
Index 814 extracted
Start decoding Index 816
Index 816 extracted
Start decoding Index 817
Index 817 extracted
Start decoding Index 819
Index 819 extracted
Start decoding Index 820
Index 820 extracted
Start decoding Index 822
Index 822 

Index 1129 extracted
Start decoding Index 1131
Index 1131 extracted
Start decoding Index 1133
Index 1133 extracted
Start decoding Index 1135
Index 1135 extracted
Start decoding Index 1146
Index 1146 extracted
Start decoding Index 1147
Index 1147 extracted
Start decoding Index 1149
Index 1149 extracted
Start decoding Index 1150
Index 1150 extracted
Start decoding Index 1152
Index 1152 extracted
Start decoding Index 1153
Index 1153 extracted
Start decoding Index 1155
Index 1155 extracted
Start decoding Index 1156
Index 1156 extracted
Start decoding Index 1158
Index 1158 extracted
Start decoding Index 1159
Index 1159 extracted
Start decoding Index 1161
Index 1161 extracted
Start decoding Index 1162
Index 1162 extracted
Start decoding Index 1164
Index 1164 extracted
Start decoding Index 1165
Index 1165 extracted
Start decoding Index 1167
Index 1167 extracted
Start decoding Index 1168
Index 1168 extracted
Start decoding Index 1170
Index 1170 extracted
Start decoding Index 1171
Index 1171 ex

...

Index 20852 extracted
Start decoding Index 20854
Index 20854 extracted
Start decoding Index 20856
Index 20856 extracted
Start decoding Index 20858
Index 20858 extracted
Start decoding Index 20860
Index 20860 extracted
Start decoding Index 20862
Index 20862 extracted
Start decoding Index 20864
Index 20864 extracted
Start decoding Index 20866
Index 20866 extracted
Start decoding Index 20868
Index 20868 extracted
Start decoding Index 20870
Index 20870 extracted
Start decoding Index 20872
Index 20872 extracted
Start decoding Index 20874
Index 20874 extracted
Start decoding Index 20876
Index 20876 extracted
Start decoding Index 20878
Index 20878 extracted
Start decoding Index 20880
Index 20880 extracted
Start decoding Index 20882
Index 20882 extracted
Start decoding Index 20884
Index 20884 extracted
Start decoding Index 20886
Index 20886 extracted
Start decoding Index 20898
Index 20898 extracted
Start decoding Index 20899
Index 20899 extracted
Start decoding Index 20901
Index 20901 extracte

Index 21197 extracted
Start decoding Index 21199
Index 21199 extracted
Start decoding Index 21200
Index 21200 extracted
Start decoding Index 21203
Index 21203 extracted
Start decoding Index 21204
Index 21204 extracted
Start decoding Index 21206
Index 21206 extracted
Start decoding Index 21207
Index 21207 extracted
Start decoding Index 21209
Index 21209 extracted
Start decoding Index 21210
Index 21210 extracted
Start decoding Index 21212
Index 21212 extracted
Start decoding Index 21213
Index 21213 extracted
Start decoding Index 21215
Index 21215 extracted
Start decoding Index 21216
Index 21216 extracted
Start decoding Index 21218
Index 21218 extracted
Start decoding Index 21219
Index 21219 extracted
Start decoding Index 21221
Index 21221 extracted
Start decoding Index 21222
Index 21222 extracted
Start decoding Index 21224
Index 21224 extracted
Start decoding Index 21225
Index 21225 extracted
Start decoding Index 21227
Index 21227 extracted
Start decoding Index 21228
Index 21228 extracte

Index 22051 extracted
Start decoding Index 22053
Index 22053 extracted
Start decoding Index 22055
Index 22055 extracted
Start decoding Index 22057
Index 22057 extracted
Start decoding Index 22059
Index 22059 extracted
Start decoding Index 22061
Index 22061 extracted
Start decoding Index 22063
Index 22063 extracted
Start decoding Index 22065
Index 22065 extracted
Start decoding Index 22067
Index 22067 extracted
Start decoding Index 22069
Index 22069 extracted
Start decoding Index 22080
Index 22080 extracted
Start decoding Index 22081
Index 22081 extracted
Start decoding Index 22083
Index 22083 extracted
Start decoding Index 22084
Index 22084 extracted
Start decoding Index 22086
Index 22086 extracted
Start decoding Index 22087
Index 22087 extracted
Start decoding Index 22089
Index 22089 extracted
Start decoding Index 22090
Index 22090 extracted
Start decoding Index 22092
Index 22092 extracted
Start decoding Index 22093
Index 22093 extracted
Start decoding Index 22095
Index 22095 extracte

Index 22333 extracted
Start decoding Index 22334
Index 22334 extracted
Start decoding Index 22336
Index 22336 extracted
Start decoding Index 22337
Index 22337 extracted
Start decoding Index 22339
Index 22339 extracted
Start decoding Index 22340
Index 22340 extracted
Start decoding Index 22342
Index 22342 extracted
Start decoding Index 22343
Index 22343 extracted
Start decoding Index 22345
Index 22345 extracted
Start decoding Index 22346
Index 22346 extracted
Start decoding Index 22348
Index 22348 extracted
Start decoding Index 22349
Index 22349 extracted
Start decoding Index 22351
Index 22351 extracted
Start decoding Index 22352
Index 22352 extracted
Start decoding Index 22354
Index 22354 extracted
Start decoding Index 22355
Index 22355 extracted
Start decoding Index 22357
Index 22357 extracted
Start decoding Index 22358
Index 22358 extracted
Start decoding Index 22370
Index 22370 extracted
Start decoding Index 22372
Index 22372 extracted
Start decoding Index 22374
Index 22374 extracte

Index 22713 extracted
Start decoding Index 22724
Index 22724 extracted
Start decoding Index 22725
Index 22725 extracted
Start decoding Index 22727
Index 22727 extracted
Start decoding Index 22728
Index 22728 extracted
Start decoding Index 22730
Index 22730 extracted
Start decoding Index 22731
Index 22731 extracted
Start decoding Index 22733
Index 22733 extracted
Start decoding Index 22734
Index 22734 extracted
Start decoding Index 22736
Index 22736 extracted
Start decoding Index 22737
Index 22737 extracted
Start decoding Index 22739
Index 22739 extracted
Start decoding Index 22740
Index 22740 extracted
Start decoding Index 22742
Index 22742 extracted
Start decoding Index 22743
Index 22743 extracted
Start decoding Index 22745
Index 22745 extracted
Start decoding Index 22746
Index 22746 extracted
Start decoding Index 22748
Index 22748 extracted
Start decoding Index 22749
Index 22749 extracted
Start decoding Index 22763
Index 22763 extracted
Start decoding Index 22764
Index 22764 extracte

In [ ]:
data_et = convertToNumeric(data_et, ['x', 'y', 't', 't_task'])

In [ ]:
data_et.groupby(['run_id', 'trial_index'])['x', 'y'].count()

# data_subject

In [ ]:
data_subject = data_trial.loc[
    : ,
    [
        'chinFirst', 'choiceTask_amountLeftFirst', 'chosenAmount',
        'chosenDelay', 'run_id', 'prolificID'
    ]
] \
    .merge(data_raw.loc[:, ['run_id',
                            'age', 'browliner', 'browser', 'browser_version', 'degree', 'device',
                            'ethnic', 'eyeliner', 'eyeshadow', 'gender', 'glasses', 'keptHead',
                            'masquara', 'optionalNote', 'platform', 'platform_version', 'sight',
                            'triedChin', 'user_agent', 'vertPosition', 'webcam_fps',
                            'webcam_height', 'webcam_label', 'webcam_width']],
           on='run_id', 
           how='left') \
    .drop_duplicates()
data_subject.columns

# Export data

In [ ]:
if not os.path.exists('./data_jupyter'):
    os.mkdir('./data_jupyter')

data_et.to_csv("data_jupyter/data_et.csv", index=False, header=True)
data_trial.to_csv("data_jupyter/data_trial.csv", index=False, header=True)
data_subject.to_csv("data_jupyter/data_subject.csv", index=False, header=True)

# Feedback

In [ ]:
print('Success! Script ran through')